# Simulate LFP Correlation

This notebook...

## Imports

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import neo

import os

print("Packages Imported!")

Packages Imported!


## Settings

In [5]:
PROJECT_PATH = 'G:/Shared drives/visual_encoding'

## Load Data

In [6]:
# load blocks, store data

for i, file in enumerate(os.listdir("D:/visual_encoding/data/blocks/segmented/by_cell_type/")):
    
    block = neo.io.NeoMatlabIO(f"D:/visual_encoding/data/blocks/segmented/by_cell_type/{file}").read_block()
    
    break

In [7]:
block

Block with 20 segments
annotations: {'session_id': '847657808',
  'spike_brain_structures': 'VISp',
  'stimulus_name': 'natural_movie_one_shuffled',
  'frame': 0,
  'running': array([ True,  True,  True,  True, False, False, False, False, False,
         False,  True,  True, False, False, False, False, False, False,
         False, False])}
# segments (N=20)
0: Segment with 2 analogsignals, 50 spiketrains
   annotations: {'stimulus_onset': 3191.2397299547883,
     'time_window': array([ 0, 30]),
     'running': True}
   # analogsignals (N=2)
   0: AnalogSignal with 1 channels of length 1500; units cm/s; datatype float64 
      name: 'running_speed'
      sampling rate: 50.0 Hz
      time: 3191.2400000000002 s to 3221.2400000000002 s
   1: AnalogSignal with 1 channels of length 1500; units cm**2; datatype float64 
      name: 'pupil_area'
      sampling rate: 50.0 Hz
      time: 3191.2400000000002 s to 3221.2400000000002 s
1: Segment with 2 analogsignals, 50 spiketrains
   annotations: 

## Main

In [ ]:
# filter spikes by cell type

In [ ]:
# convolve with synaptic kernels + sum


In [ ]:
# plot examples with actual LFP side by side


In [ ]:
# compute correlation for each pair
